<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/274_MissionOrchestratorAgent_Toolshed_HITL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human-in-the-Loop (HITL) Utilities

In [ ]:
"""
Human-in-the-Loop (HITL) Utilities

Reusable utilities for handling human approval workflows in orchestrator agents.
"""

from toolshed.hitl.approval_requests import (
    check_task_requires_approval,
    create_approval_request
)
from toolshed.hitl.approval_processing import (
    process_approval,
    auto_approve_for_testing
)
from toolshed.hitl.approval_tracking import (
    get_pending_approvals,
    is_task_approved,
    find_tasks_requiring_approval
)

__all__ = [
    "check_task_requires_approval",
    "create_approval_request",
    "process_approval",
    "auto_approve_for_testing",
    "get_pending_approvals",
    "is_task_approved",
    "find_tasks_requiring_approval"
]




# ✅ Approval Request Creation & Management — Summary

These utilities handle the **moment a task finishes and the agent needs to ask a human for approval** before continuing.

If the previous utilities handled *recording* approvals after a human decides,
these utilities handle *creating* the approval requests in the first place.

Think of this as the agent raising its hand and saying:

> “I’m done with this step — can someone check my work?”

---

# ⭐ `check_task_requires_approval(task)`

This is a simple helper that answers one question:

> “Does this particular task need a human to check it?”

It looks for the field:

```python
"requires_human_approval": True/False
```

Why it matters:

* Not every task needs approval
* The orchestrator should only pause for tasks that require it
* Keeps the workflow fast and efficient

This lets your nodes quickly filter:

* tasks that need to go through HITL
* tasks that can run automatically

Clean, simple, essential.

---

# ⭐ `create_approval_request(task_result)`

This function builds the **approval request object** the orchestrator uses to track pending approvals.

It includes:

* `task_id`
* task description
* agent name
* full execution result
* timestamp of when the approval was requested
* status: `"pending"`

In human terms:

> It creates a “Please review this” slip with all the important details attached.

Why it's important:

* The orchestrator needs a formal request to show humans
* The request becomes part of the audit trail
* Progress nodes and completion check nodes use these requests
* The UI or notifications layer can use these objects directly

This function ensures every approval request has:

* consistent structure
* proper timestamps
* all necessary info for a human reviewer

So the system is predictable and easy to integrate with future interfaces (Slack, dashboards, etc.).

---

# 🎯 Big Picture

These utilities handle the *front half* of HITL:

### Before a human approves:

* Check if approval is required
* Generate a request object
* Mark it as pending
* Include all the needed details

The previous utilities handled the *back half*:

### After the human approves:

* Record the decision
* Add it to approval history
* Allow the workflow to proceed

Together, they form a complete HITL subsystem:

* identify
* request
* wait
* approve
* move on

This subsystem is clean, reusable, and foundational to real-world orchestrators where humans stay in control of key decisions.



In [ ]:
"""Approval request creation and management utilities"""

from typing import Dict, Any, Optional
from datetime import datetime


def check_task_requires_approval(task: Dict[str, Any]) -> bool:
    """
    Check if a task requires human approval.

    Args:
        task: Task dictionary with 'requires_human_approval' field

    Returns:
        True if task requires approval, False otherwise
    """
    return task.get("requires_human_approval", False)


def create_approval_request(
    task_result: Dict[str, Any],
    requested_at: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create an approval request for a task.

    Args:
        task_result: Task execution result
        requested_at: ISO timestamp (defaults to now)

    Returns:
        Approval request dictionary with:
        - task_id: Task identifier
        - task: Task description
        - agent_name: Agent that executed the task
        - result: Task execution result
        - requested_at: Timestamp when approval was requested
        - status: "pending"
    """
    if requested_at is None:
        requested_at = datetime.now().isoformat()

    return {
        "task_id": task_result.get("task_id"),
        "task": task_result.get("task", ""),
        "agent_name": task_result.get("agent_name", ""),
        "result": task_result.get("result"),
        "requested_at": requested_at,
        "status": "pending"
    }




# ✅ Approval Processing Utilities Summary

These utilities handle the part of your orchestrator where a **human must approve a task** before the workflow can continue.
Think of it as the agent saying:

> “Hey, I’m done with this step — is it OK for me to move on?”

The toolshed contains two key utilities for that moment.

---

## ⭐ `process_approval()`

This function creates a **clean, structured record** of a human’s decision about a task.

It records:

* the task ID
* whether it was approved or rejected
* who approved it
* when they approved it
* when the approval was originally requested

Why this matters:

* The orchestrator must know exactly what was approved
* It keeps a trustworthy audit trail
* It allows later nodes (like completion checks) to make decisions confidently
* It makes reports accurate and transparent

In short:

> It turns a human decision into a machine-readable receipt.

If the decision isn’t `"approved"` or `"rejected"`, it raises an error, ensuring the workflow stays safe and predictable.

---

## ⭐ `auto_approve_for_testing()`

When you’re developing or testing agents, having to stop and manually approve tasks would be annoying and slow.
This function solves that.

It:

* loops through all pending approval requests
* automatically approves each one
* marks the decision as `"approved"` by `"auto_approval"`

Why this is helpful:

* lets your orchestrator run start-to-finish without stopping
* perfect for debugging and demos
* ensures HITL features don’t interrupt testing

But in production:

* you turn this off
* real humans enter real approvals
* the orchestrator waits properly

In short:

> It’s a temporary shortcut that keeps development smooth.

---

# 🎯 The Big Picture

Together, these utilities handle:

* **Recording human decisions**
* **Skipping human decisions during testing**
* **Maintaining auditability**
* **Keeping the orchestrator safe and compliant**

And because they’re in your toolshed:

* you can reuse them in *any* agent
* you never have to rewrite approval logic
* you can expand them later (Slack approvals, email approvals, UI-based approvals, etc.)

This is foundational infrastructure for *every real-world automation system*.




In [ ]:
"""Approval processing utilities"""

from typing import List, Dict, Any, Optional
from datetime import datetime


def process_approval(
    approval_request: Dict[str, Any],
    decision: str,
    decided_by: Optional[str] = None,
    decided_at: Optional[str] = None
) -> Dict[str, Any]:
    """
    Process an approval decision.

    Args:
        approval_request: Approval request dictionary
        decision: "approved" or "rejected"
        decided_by: Human identifier (defaults to "human")
        decided_at: ISO timestamp (defaults to now)

    Returns:
        Approval history entry with:
        - task_id: Task identifier
        - decision: "approved" or "rejected"
        - decided_at: Timestamp of decision
        - decided_by: Who made the decision
        - requested_at: Original request timestamp

    Raises:
        ValueError: If decision is not "approved" or "rejected"
    """
    if decided_by is None:
        decided_by = "human"

    if decided_at is None:
        decided_at = datetime.now().isoformat()

    if decision not in ["approved", "rejected"]:
        raise ValueError(f"Invalid decision: {decision}. Must be 'approved' or 'rejected'")

    return {
        "task_id": approval_request.get("task_id"),
        "decision": decision,
        "decided_at": decided_at,
        "decided_by": decided_by,
        "requested_at": approval_request.get("requested_at")
    }


def auto_approve_for_testing(
    pending_approvals: List[Dict[str, Any]],
    auto_approve: bool = True
) -> List[Dict[str, Any]]:
    """
    Auto-approve all pending approvals (for testing purposes).

    Args:
        pending_approvals: List of pending approval requests
        auto_approve: Whether to auto-approve (default: True)

    Returns:
        List of approval history entries (empty if auto_approve is False)
    """
    if not auto_approve:
        return []

    approvals = []
    for request in pending_approvals:
        approval = process_approval(request, "approved", decided_by="auto_approval")
        approvals.append(approval)

    return approvals




# ✅ Approval Tracking & Status Utilities — Summary

Once your orchestrator knows:

* which tasks require approval
* how to create an approval request
* how to record a final approval decision

…it still needs a way to **keep track of which tasks are pending and which are finished.**

That’s where these utilities come in.

Think of them as the agent’s way of asking:

> “Okay, what’s still waiting on a human?”
> “What’s done?”
> “Who signed off already?”

These functions form the monitoring layer of HITL.

---

## ⭐ `get_pending_approvals()`

This is the **main function** of this group.

It figures out which tasks:

* have been executed
* require approval
* haven’t been approved yet

And for each of those, it creates a **fresh approval request object**.

Why this matters:

* The orchestrator needs a current list of “unapproved tasks”
* Approval check nodes use this list to decide what to do next
* It allows accurate mission status transitions (in_progress → awaiting_approval)
* It keeps approval tracking consistent and automatic

In simple terms:

> “Show me everything the agent did that still needs a human signature.”

This is critical for workflow control.

---

## ⭐ `is_task_approved(task_id, approval_history)`

A tiny helper, but extremely important.

It checks:

> “Has this specific task already been approved?”

It looks through the approval history and returns `True` or `False`.

Why it’s valuable:

* Nodes can quickly check approval status
* Prevents duplicate requests
* Keeps logic simple throughout the orchestrator
* Supports future UI/workflow integrations

This function is foundational for any conditional routing or validation.

---

## ⭐ `find_tasks_requiring_approval()`

This is similar to `get_pending_approvals`, but works on **raw executed task results**.

It answers:

> “Among the tasks we've completed, which ones *say* they require approval but don’t have an `approved` flag yet?”

Unlike `get_pending_approvals`, this one does **not** generate approval request objects — it returns the raw task results.

Why this exists:

* Useful for early HITL detection
* Useful in debugging and testing
* Useful when approval request objects aren’t needed yet

It’s essentially a filtered list of “approval-needed tasks.”

---

# 🎯 Big Picture

These utilities form the **tracking layer** of HITL — the part that ensures:

* nothing requiring approval is missed
* nothing gets approved twice
* nothing blocks the workflow incorrectly
* the orchestrator always knows whether it can continue or must wait

Together, they support:

### ✔ the approval check node

### ✔ the completion check node

### ✔ reporting

### ✔ general state awareness

You can think of them as the "approval traffic reporters" for the whole system:

* They tell you what's waiting
* They tell you what's done
* They tell you what needs attention

This keeps your orchestrator honest, predictable, and rock-solid when humans are part of the workflow.




In [ ]:
"""Approval tracking and status utilities"""

from typing import List, Dict, Any


def get_pending_approvals(approval_history: List[Dict[str, Any]], executed_tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Get list of tasks with pending approvals.

    Args:
        approval_history: List of approval decisions
        executed_tasks: List of executed tasks

    Returns:
        List of pending approval requests
    """
    approved_task_ids = {
        entry.get("task_id") for entry in approval_history
        if entry.get("decision") == "approved"
    }

    pending = []

    for task_result in executed_tasks:
        task_id = task_result.get("task_id")

        # Task requires approval if:
        # 1. It's completed
        # 2. It requires approval (from original task definition)
        # 3. It hasn't been approved yet
        if (task_result.get("status") == "completed" and
            task_result.get("requires_approval", False) and
            task_id not in approved_task_ids):

            from toolshed.hitl.approval_requests import create_approval_request
            approval_request = create_approval_request(task_result)
            pending.append(approval_request)

    return pending


def is_task_approved(task_id: str, approval_history: List[Dict[str, Any]]) -> bool:
    """
    Check if a task has been approved.

    Args:
        task_id: Task identifier
        approval_history: List of approval decisions

    Returns:
        True if task is approved, False otherwise
    """
    for entry in approval_history:
        if entry.get("task_id") == task_id and entry.get("decision") == "approved":
            return True
    return False


def find_tasks_requiring_approval(executed_tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Find tasks that require approval and haven't been approved yet.

    Args:
        executed_tasks: List of executed task results

    Returns:
        List of tasks that need approval
    """
    pending = []

    for task_result in executed_tasks:
        task_id = task_result.get("task_id")
        status = task_result.get("status")

        # Check if task requires approval and is completed but not yet approved
        if status == "completed" and task_result.get("requires_approval", False):
            # Check if already approved
            if not task_result.get("approved", False):
                pending.append(task_result)

    return pending

